# Data preprocessing 

Here we will import the data and process it. This will make it possible to do further analysis later on

# Import

In [4]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import itertools
import numpy as np
import warnings

In [5]:
warnings.simplefilter('ignore')

# Raw data

We mined twitter data about the COVID-19 virus from febuary 23 until march 15 using .....py. We also gathered COVID-19 Statistics from....

### Mined data

In [6]:
# get all data files and save them in a list FilenNames
Mypath = "Data/Tweets"
FileNames = [f for f in listdir(Mypath) if isfile(join(Mypath, f))]
print(FileNames)
# Each day has its own csv file
print(len(FileNames))

['output_ALL_01-mar.csv', 'output_ALL_02-mar.csv', 'output_ALL_03-mar.csv', 'output_ALL_04-mar.csv', 'output_ALL_05-mar.csv', 'output_ALL_06-mar.csv', 'output_ALL_07-mar.csv', 'output_ALL_08-mar.csv', 'output_ALL_09-mar.csv', 'output_ALL_10-mar.csv', 'output_ALL_11-mar.csv', 'output_ALL_12-mar.csv', 'output_ALL_13-mar.csv', 'output_ALL_14-mar.csv', 'output_ALL_15-mar.csv', 'output_ALL_23-feb.csv', 'output_ALL_24-feb.csv', 'output_ALL_25-feb.csv', 'output_ALL_26-feb.csv', 'output_ALL_27-feb.csv', 'output_ALL_28-feb.csv', 'output_ALL_29-feb.csv']
22


In [7]:
# DataFrame of all the coronavirus tweets on 27/02/2020
feb27 = pd.read_csv("Data/Tweets/output_ALL_07-mar.csv",error_bad_lines=False, sep=';')
feb27.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink,Unnamed: 10
0,NaN,07/03/2020 23:59,0,3,# NRC Dit krantje vergelijkt het # coronavirus...,NaN,NaN,# # #,"1,24E+18",https://twitter.com/HellingsTony/status/123642...,NaN
1,NaN,07/03/2020 23:59,0,1,"Maar mensen van 10, 20 of 30 jaar zouden toch ...",NaN,NaN,NaN,"1,24E+18",https://twitter.com/VrijdenkerRitch/status/123...,NaN
2,NaN,07/03/2020 23:58,0,5,Wat staat er morgen op de agenda: de nieuwe pe...,NaN,@ @,# #,"1,24E+18",https://twitter.com/Eliezeryair/status/1236426...,NaN
3,NaN,07/03/2020 23:58,0,0,Iedereen is hier welkom. Ook het corona virus ...,NaN,NaN,NaN,"1,24E+18",https://twitter.com/Anti_S0cialist/status/1236...,NaN
4,NaN,07/03/2020 23:57,0,2,De vier studenten van de Groningse studentenve...,NaN,NaN,#,"1,24E+18",https://twitter.com/Robbedoes1999/status/12364...,NaN


In [8]:
# List with a DataFrame of tweets for each day 
pdList = [pd.read_csv(Mypath+"/"+x,error_bad_lines=False, sep=';') for x in FileNames]

b'Skipping line 2448: expected 10 fields, saw 11\nSkipping line 3057: expected 10 fields, saw 11\n'
b'Skipping line 4418: expected 10 fields, saw 11\n'
b'Skipping line 4819: expected 10 fields, saw 11\n'
b'Skipping line 4532: expected 10 fields, saw 11\nSkipping line 6040: expected 10 fields, saw 11\nSkipping line 6594: expected 10 fields, saw 11\nSkipping line 6750: expected 10 fields, saw 11\n'
b'Skipping line 1573: expected 10 fields, saw 12\nSkipping line 2320: expected 10 fields, saw 11\nSkipping line 6616: expected 10 fields, saw 11\n'
b'Skipping line 858: expected 10 fields, saw 11\nSkipping line 1094: expected 10 fields, saw 11\nSkipping line 4910: expected 10 fields, saw 11\nSkipping line 4965: expected 10 fields, saw 11\nSkipping line 6911: expected 10 fields, saw 11\nSkipping line 8897: expected 10 fields, saw 11\nSkipping line 10038: expected 10 fields, saw 11\nSkipping line 10093: expected 10 fields, saw 11\n'
b'Skipping line 269: expected 10 fields, saw 11\nSkipping line 

In [9]:
# combining DataFrames into one DataFrame containing all tweets
AllTweets = pd.concat(pdList, ignore_index=0)

In [10]:
# Drop unused columns
AllTweets = AllTweets.drop(columns=["Unnamed: 10","Unnamed: 11"])

In [11]:
AllTweets = AllTweets.reset_index()

We mined more then 165,000 tweets

#### Save as .csv file

In [12]:
# Save dataframe as csv to the Data folder
AllTweets.to_csv("Data/AllTweets.csv")

### COVID-19 Statistics

In [77]:
# confirmed cases deaths and recoveries saved in their own dataframe
Cases = pd.read_csv("Data/Cases/confirmedCases.csv").transpose()
Cases = Cases.rename(columns=Cases.iloc[0]).drop(Cases.index[0]).reset_index()
Death = pd.read_csv("Data/Cases/confirmedDeaths.csv").transpose()
Death = Death.rename(columns=Death.iloc[0]).drop(Death.index[0]).reset_index()
Recoveries = pd.read_csv("Data/Cases/confirmedRecoveries.csv").transpose() 
Recoveries = Recoveries.rename(columns=Recoveries.iloc[0]).drop(Recoveries.index[0])

In [78]:
Cases.head()

,index,Netherlands,Europe,EU,Non-Europe,Worldwide
0,1/22/20,0,0,0,555,555
1,1/23/20,0,0,0,653,653
2,1/24/20,0,2,2,939,941
3,1/25/20,0,3,3,1431,1434
4,1/26/20,0,3,3,2115,2118


# Processing

## Counting amount of tweets per hour

In [79]:
def TweetCounter(FileName):
    """
    Counts the tweets per hour for the input file and returns a Dataframe with the timestamp and the amount of tweets in that hour
    """
    FolderName = "Data/Tweets"
    df = pd.read_csv(FolderName+"/"+FileName,error_bad_lines=False, sep=';')
    output = pd.DataFrame(columns=["datetime","coronavirus"])
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    date = str(df["date"].iloc[-1]).split(" ")[0]
    for Hour in range(24):
        tempDF = df[(df["date"].apply(lambda x : x.hour)==Hour)]
        time = str(Hour)+":00:00"
        datetime = date + " " + time
        output.loc[Hour] = [datetime, tempDF.date.count()]
    return output  

In [80]:
TweetCounter("output_ALL_13-mar.csv")

,datetime,coronavirus
0,2020-03-13 0:00:00,0
1,2020-03-13 1:00:00,207
2,2020-03-13 2:00:00,106
3,2020-03-13 3:00:00,87
4,2020-03-13 4:00:00,40
5,2020-03-13 5:00:00,74
6,2020-03-13 6:00:00,233
7,2020-03-13 7:00:00,537
8,2020-03-13 8:00:00,876
9,2020-03-13 9:00:00,1092


#### Save as .csv file

In [81]:
# Save the tweets per hour dataframes as csv files in the Data/PerHour folder
for file in FileNames:
    fd2 = TweetCounter(file)
    fd2.to_csv("Data/PerHour/"+file)

b'Skipping line 2448: expected 10 fields, saw 11\nSkipping line 3057: expected 10 fields, saw 11\n'
b'Skipping line 4418: expected 10 fields, saw 11\n'
b'Skipping line 4819: expected 10 fields, saw 11\n'
b'Skipping line 4532: expected 10 fields, saw 11\nSkipping line 6040: expected 10 fields, saw 11\nSkipping line 6594: expected 10 fields, saw 11\nSkipping line 6750: expected 10 fields, saw 11\n'
b'Skipping line 1573: expected 10 fields, saw 12\nSkipping line 2320: expected 10 fields, saw 11\nSkipping line 6616: expected 10 fields, saw 11\n'
b'Skipping line 858: expected 10 fields, saw 11\nSkipping line 1094: expected 10 fields, saw 11\nSkipping line 4910: expected 10 fields, saw 11\nSkipping line 4965: expected 10 fields, saw 11\nSkipping line 6911: expected 10 fields, saw 11\nSkipping line 8897: expected 10 fields, saw 11\nSkipping line 10038: expected 10 fields, saw 11\nSkipping line 10093: expected 10 fields, saw 11\n'
b'Skipping line 269: expected 10 fields, saw 11\nSkipping line 

### Combining data

In [82]:
# get data saved in the per hour format from Data/PerHour and combines it into one DataFrame
Mypath = "Data/PerHour"
FileNames = [f for f in listdir(Mypath) if isfile(join(Mypath, f))]
pdList = [pd.read_csv(Mypath+"/"+x,error_bad_lines=False, index_col=0) for x in FileNames]
TwitterCovPerHour = pdList[0]
for x in range(1,len(pdList)):
    TwitterCovPerHour = TwitterCovPerHour.merge(pdList[x], how="outer")

In [83]:
TwitterCovPerHour

,datetime,coronavirus
0,2020-03-01 0:00:00,76
1,2020-03-01 1:00:00,46
2,2020-03-01 2:00:00,31
3,2020-03-01 3:00:00,21
4,2020-03-01 4:00:00,16
...,...,...
523,2020-02-29 19:00:00,223
524,2020-02-29 20:00:00,121
525,2020-02-29 21:00:00,216
526,2020-02-29 22:00:00,250


### Splitting columns

In [84]:
# Splitting the datetime collum into two new colums one with the date and one with the time 
# and also keeping the original datetime column
new = TwitterCovPerHour["datetime"].str.split(" ", n=1, expand = True)
TwitterCovPerHour["datetime"] = TwitterCovPerHour["datetime"]
TwitterCovPerHour["Date"] = new[0]
TwitterCovPerHour["Time"] = new[1]
TwitterCovPerHour
TwitterCovPerHour["datetime"] = pd.to_datetime(TwitterCovPerHour["datetime"], dayfirst=True )

In [85]:
TwitterCovPerHour

,datetime,coronavirus,Date,Time
0,2020-03-01 00:00:00,76,2020-03-01,0:00:00
1,2020-03-01 01:00:00,46,2020-03-01,1:00:00
2,2020-03-01 02:00:00,31,2020-03-01,2:00:00
3,2020-03-01 03:00:00,21,2020-03-01,3:00:00
4,2020-03-01 04:00:00,16,2020-03-01,4:00:00
...,...,...,...,...
523,2020-02-29 19:00:00,223,2020-02-29,19:00:00
524,2020-02-29 20:00:00,121,2020-02-29,20:00:00
525,2020-02-29 21:00:00,216,2020-02-29,21:00:00
526,2020-02-29 22:00:00,250,2020-02-29,22:00:00


### Sorting dataframe by datetime

In [86]:
TwitterCovPerHour = TwitterCovPerHour.sort_values(by="datetime", ascending=True)
TwitterCovPerHour = TwitterCovPerHour.reset_index(drop=True)

In [87]:
TwitterCovPerHour

,datetime,coronavirus,Date,Time
0,2020-02-23 00:00:00,54,2020-02-23,0:00:00
1,2020-02-23 01:00:00,21,2020-02-23,1:00:00
2,2020-02-23 02:00:00,12,2020-02-23,2:00:00
3,2020-02-23 03:00:00,11,2020-02-23,3:00:00
4,2020-02-23 04:00:00,10,2020-02-23,4:00:00
...,...,...,...,...
523,2020-03-15 19:00:00,1178,2020-03-15,19:00:00
524,2020-03-15 20:00:00,981,2020-03-15,20:00:00
525,2020-03-15 21:00:00,738,2020-03-15,21:00:00
526,2020-03-15 22:00:00,749,2020-03-15,22:00:00


### Cumulative of the amount of tweets per hour

In [88]:
# Adding an extra column containing the cumulative of the amount of tweets
TwitterCovPerHour["CumSum"] = TwitterCovPerHour.coronavirus.cumsum(axis=None, skipna=True)

In [89]:
TwitterCovPerHour.head()

,datetime,coronavirus,Date,Time,CumSum
0,2020-02-23 00:00:00,54,2020-02-23,0:00:00,54
1,2020-02-23 01:00:00,21,2020-02-23,1:00:00,75
2,2020-02-23 02:00:00,12,2020-02-23,2:00:00,87
3,2020-02-23 03:00:00,11,2020-02-23,3:00:00,98
4,2020-02-23 04:00:00,10,2020-02-23,4:00:00,108


#### Save as csv file

In [90]:
TwitterCovPerHour.to_csv("Data/CoronaTweetsPerHour.csv")

## Counting amount of tweets per day

In [91]:
def PerDay(DataFrame):
    """
    Takes the amount of tweets per hour dataframe and turns it into the amount per day dataframe
    """
    output = pd.DataFrame(columns=["date", "tweets"])
    Dates = DataFrame["Date"].unique()
    for x in range(len(Dates)):
        TotTweets = sum(DataFrame["coronavirus"][DataFrame["Date"]== Dates[x]])
        output.loc[x] = [Dates[x], TotTweets]
        
    return output

In [92]:
TwitterCovPerDay = PerDay(TwitterCovPerHour)

In [93]:
TwitterCovPerDay.head()

,date,tweets
0,2020-02-23,1471
1,2020-02-24,3121
2,2020-02-25,3479
3,2020-02-26,4831
4,2020-02-27,7122


### Cumulative of the amount of tweets per day

In [94]:
# Adding an extra column containing the cumulative of the amount of tweets per day
TwitterCovPerDay["CumSum"] = TwitterCovPerDay.tweets.cumsum(axis=None, skipna=True)

In [95]:
TwitterCovPerDay.head()

,date,tweets,CumSum
0,2020-02-23,1471,1471
1,2020-02-24,3121,4592
2,2020-02-25,3479,8071
3,2020-02-26,4831,12902
4,2020-02-27,7122,20024


#### Save as csv file

In [96]:
TwitterCovPerDay.to_csv("Data/CoronaTweetsPerDay.csv")

## Normalize data

In [97]:
def normalize(df):
    """
    Normalize columns in dataframe and returns a dataframe
    """
    result = df.copy()
    for feature_name in df.columns[1:]:
        max_value = int(df[feature_name].max())
        min_value = int(df[feature_name].min())
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

### Normalize tweets per day

In [98]:
NormalizedDFPerDay = normalize(TwitterCovPerDay)

In [99]:
NormalizedDFPerDay.tail()

,date,tweets,CumSum
17,2020-03-11,0.469868,0.58643
18,2020-03-12,1,0.70908
19,2020-03-13,0.885595,0.818726
20,2020-03-14,0.645988,0.901136
21,2020-03-15,0.79074,1


#### Save as csv file

In [100]:
NormalizedDFPerDay.to_csv('Data/NormalizedDFPerDay.csv')

### Normalize tweets per hour

In [101]:
NormalizedDFPerHour = normalize(TwitterCovPerHour[["datetime","coronavirus", "CumSum"]])

In [102]:
NormalizedDFPerHour

,datetime,coronavirus,CumSum
0,2020-02-23 00:00:00,0.026226,0.000000
1,2020-02-23 01:00:00,0.010199,0.000127
2,2020-02-23 02:00:00,0.005828,0.000200
3,2020-02-23 03:00:00,0.005342,0.000266
4,2020-02-23 04:00:00,0.004857,0.000327
...,...,...,...
523,2020-03-15 19:00:00,0.572122,0.981883
524,2020-03-15 20:00:00,0.476445,0.987821
525,2020-03-15 21:00:00,0.358426,0.992289
526,2020-03-15 22:00:00,0.363769,0.996822


#### Save as csv file

In [103]:
NormalizedDFPerHour.to_csv('Data/NormalizedDFPerHour.csv')

## New COVID-19 cases per day

In [104]:
# new cases per day
NewCasesPerDay = pd.DataFrame(columns = ['date','Netherlands','Europe', 'EU','Non-Europe','Worldwide'])

for RowIndex in range(1,len(Cases)):
    WorldWide = Cases.Worldwide.loc[RowIndex] - Cases.Worldwide.loc[RowIndex-1]
    NonEurope = Cases["Non-Europe"].loc[RowIndex] - Cases["Non-Europe"].loc[RowIndex-1]
    Eu = Cases.EU.loc[RowIndex] - Cases.EU.loc[RowIndex-1]
    Europe = Cases.Europe.loc[RowIndex] - Cases.Europe.loc[RowIndex-1]
    Netherlands = Cases.Netherlands.loc[RowIndex] - Cases.Netherlands.loc[RowIndex-1]
    Date = Cases['index'].loc[RowIndex]
    NewRow = [Date, Netherlands, Europe, Eu, NonEurope, WorldWide]
    
    NewCasesPerDay.loc[RowIndex] = NewRow

In [106]:
NewCasesPerDay.tail()

,date,Netherlands,Europe,EU,Non-Europe,Worldwide
49,3/11/20,121,5020,4520,2253,7273
50,3/12/20,0,725,566,1753,2478
51,3/13/20,301,14078,12794,2772,16850
52,3/14/20,155,8165,7397,2739,10904
53,3/15/20,176,8343,7278,3009,11352


#### Save as csv file

In [107]:
NewCasesPerDay.to_csv('Data/NewCasesPerDay.csv')